In [ ]:
#Import Modules
from logger import setup_custom_logger
import pandas as pd
import requests
import cbsodata
import datetime

In [ ]:
# Setup Logger
try:
    logger = setup_custom_logger("E005_Investments_Non_Current_Assets)
    logger.info('starting')
except:
    logger.exception('logger could not be loaded')
    raise

In [ ]:
try:
    # Get current date information
    now = datetime.datetime.now()
    
    logger.info('datetime loaded')
    yearMin = now.year-3
    yearMax = now.year
    
    # Values you can load
    # now.year
    # now.month
    # now.day
    # now.hour
    # now.minute
except:
    logger.exception('datetime could not be loaded')
    logger.info('set yearmin to a default value')
    # Set default values for fallback
    yearMin = 2010
    yearMax = 2030
    raise

In [ ]:
# Dataset 84106NED
dataset_id = '84106NED'

# Non-Current-Assets = Vaste Activa!

# Table definitions

# SoortMutaties: A045300 = Bruto_inversteringen_in_v

